<a href="https://colab.research.google.com/github/Retmmmy/finding-vulnerabilities/blob/master/ib_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install torch torchvision torchaudio
!pip install transformers datasets
!pip install pandas numpy scikit-learn
!pip install tree-sitter tree-sitter-c tree-sitter-cpp
!pip install wandb

import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
from datasets import load_dataset


dataset = load_dataset("DynaOuchebara/BigVul")


print("Структура датасета:", dataset)
print("\nПример одной записи:")
print(dataset['train'][0])
print("\nКолонки в датасете:", dataset['train'].column_names)

In [ ]:


import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

print("=" * 60)
print("ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ")
print("=" * 60)


dataset = load_dataset("DynaOuchebara/BigVul", split='train[:5000]')


def prepare_data_simple(dataset, max_samples=1000):
    texts, labels = [], []

    for item in dataset:
        if 'func_before' in item and 'vul' in item:
            code = item['func_before']
            label = item['vul']

            if code and isinstance(code, str):
                texts.append(code[:400])
                labels.append(label)

                if len(texts) >= max_samples:
                    break

    return texts, labels

print("Подготавливаем 500 примеров...")
texts, labels = prepare_data_simple(dataset, 500)

print(f"Загружено: {len(texts)} примеров")
print(f"Уязвимых: {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")


print("\n" + "=" * 60)
print("ЗАГРУЗКА МОДЕЛИ")
print("=" * 60)

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=2,
    id2label={0: "SAFE", 1: "VULNERABLE"},
    label2id={"SAFE": 0, "VULNERABLE": 1},
    ignore_mismatched_sizes=True
)

print(f"✅ Модель загружена! Параметров: {sum(p.numel() for p in model.parameters()):,}")


print("\nТокенизация данных...")
encodings = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)


class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

split_idx = int(len(texts) * 0.8)
train_dataset = SimpleDataset(
    {k: v[:split_idx] for k, v in encodings.items()},
    labels[:split_idx]
)
val_dataset = SimpleDataset(
    {k: v[split_idx:] for k, v in encodings.items()},
    labels[split_idx:]
)

print(f"Обучающих примеров: {len(train_dataset)}")
print(f"Валидационных примеров: {len(val_dataset)}")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


print("\n" + "=" * 60)
print("НАСТРОЙКА ОБУЧЕНИЯ")
print("=" * 60)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    push_to_hub=False,
    report_to='none',
    save_total_limit=1,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 8. ОБУЧЕНИЕ
print("\n" + "=" * 60)
print("НАЧАЛО ОБУЧЕНИЯ")
print("=" * 60)

trainer.train()

# 9. ОЦЕНКА
print("\n" + "=" * 60)
print("ОЦЕНКА МОДЕЛИ")
print("=" * 60)

results = trainer.evaluate()
print("\nРезультаты оценки:")
for key, value in results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")


print("\n" + "=" * 60)
print("ТЕСТИРОВАНИЕ НА ПРИМЕРАХ")
print("=" * 60)

test_examples = [

    """void vulnerable() {
    char buffer[10];
    gets(buffer);  // Опасная функция!
}""",


    """void safe() {
    char buffer[10];
    fgets(buffer, sizeof(buffer), stdin);  // Безопасная функция
}""",

\
    """void copy_string(char *src) {
    char dest[20];
    strcpy(dest, src);  // Опасно без проверки длины
}"""
]

print("\n🔍 Проверка модели на примерах:")

for i, code in enumerate(test_examples, 1):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, max_length=256)

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    vuln_prob = predictions[0][1].item() * 100
    safe_prob = predictions[0][0].item() * 100

    status = "🔴 УЯЗВИМЫЙ" if vuln_prob > 50 else "🟢 БЕЗОПАСНЫЙ"

    print(f"\nПример {i}:")
    print(f"Код: {code[:80]}...")
    print(f"Результат: {status}")
    print(f"Вероятность уязвимости: {vuln_prob:.1f}%")
    print(f"Вероятность безопасности: {safe_prob:.1f}%")


print("\n" + "=" * 60)
print("СОХРАНЕНИЕ МОДЕЛИ")
print("=" * 60)

save_path = "./trained_codebert_vuln_detector"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"✅ Модель сохранена в: {save_path}")



In [ ]:


print("=" * 60)
print("БАЛАНСИРОВКА ДАННЫХ")
print("=" * 60)

def prepare_balanced_data(dataset, samples_per_class=200):
    """Создает сбалансированный датасет"""
    vulnerable_texts = []
    safe_texts = []

    for item in dataset:
        if 'func_before' in item and 'vul' in item:
            code = item['func_before']
            label = item['vul']

            if code and isinstance(code, str):
                code_short = code[:400]  # Обрезаем

                if label == 1 and len(vulnerable_texts) < samples_per_class:
                    vulnerable_texts.append(code_short)
                elif label == 0 and len(safe_texts) < samples_per_class:
                    safe_texts.append(code_short)

                if len(vulnerable_texts) >= samples_per_class and len(safe_texts) >= samples_per_class:
                    break

    print(f"Уязвимых примеров: {len(vulnerable_texts)}")
    print(f"Безопасных примеров: {len(safe_texts)}")


    texts = vulnerable_texts + safe_texts
    labels = [1] * len(vulnerable_texts) + [0] * len(safe_texts)


    import random
    combined = list(zip(texts, labels))
    random.shuffle(combined)
    texts, labels = zip(*combined)

    return list(texts), list(labels)


dataset = load_dataset("DynaOuchebara/BigVul", split='train[:10000]')

print("Создаем сбалансированный датасет (по 200 каждого класса)...")
texts, labels = prepare_balanced_data(dataset, 200)

print(f"\nИтоговая статистика:")
print(f"Всего примеров: {len(texts)}")
print(f"Уязвимых (1): {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")
print(f"Безопасных (0): {len(labels)-sum(labels)} ({(len(labels)-sum(labels))/len(labels)*100:.1f}%)")



In [ ]:


import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import random

print("=" * 60)
print("ЗАГРУЗКА И БАЛАНСИРОВКА ДАННЫХ")
print("=" * 60)


dataset = load_dataset("DynaOuchebara/BigVul", split='train[:20000]')


def create_balanced_dataset(dataset, samples_per_class=250):
    """Создает идеально сбалансированный датасет"""
    vulnerable = []
    safe = []

    print("Сбор уязвимых примеров...")
    for item in dataset:
        if 'func_before' in item and 'vul' in item and item['vul'] == 1:
            code = item['func_before']
            if code and isinstance(code, str) and len(code.strip()) > 50:
                vulnerable.append(code[:500].strip())
                if len(vulnerable) >= samples_per_class:
                    break

    print("Сбор безопасных примеров...")
    for item in dataset:
        if 'func_before' in item and 'vul' in item and item['vul'] == 0:
            code = item['func_before']
            if code and isinstance(code, str) and len(code.strip()) > 50:
                safe.append(code[:500].strip())
                if len(safe) >= samples_per_class:
                    break

    print(f"\nСобрано:")
    print(f"  Уязвимых примеров: {len(vulnerable)}")
    print(f"  Безопасных примеров: {len(safe)}")


    all_texts = vulnerable + safe
    all_labels = [1] * len(vulnerable) + [0] * len(safe)


    combined = list(zip(all_texts, all_labels))
    random.shuffle(combined)
    texts, labels = zip(*combined)

    return list(texts), list(labels)


texts, labels = create_balanced_dataset(dataset, 250)

print(f"\nИтоговый датасет:")
print(f"Всего примеров: {len(texts)}")
print(f"Уязвимых: {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")
print(f"Безопасных: {len(labels)-sum(labels)} ({(len(labels)-sum(labels))/len(labels)*100:.1f}%)")


print("\n" + "=" * 60)
print("ТОКЕНИЗАЦИЯ ДАННЫХ")
print("=" * 60)


tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


print("Токенизация...")
encodings = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)


split_idx = int(len(texts) * 0.8)
train_texts, val_texts = texts[:split_idx], texts[split_idx:]
train_labels, val_labels = labels[:split_idx], labels[split_idx:]

train_encodings = {
    'input_ids': encodings['input_ids'][:split_idx],
    'attention_mask': encodings['attention_mask'][:split_idx]
}
val_encodings = {
    'input_ids': encodings['input_ids'][split_idx:],
    'attention_mask': encodings['attention_mask'][split_idx:]
}

print(f"Обучающая выборка: {len(train_texts)} примеров")
print(f"Валидационная выборка: {len(val_texts)} примеров")


print("\n" + "=" * 60)
print("СОЗДАНИЕ МОДЕЛИ")
print("=" * 60)


model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=2,
    id2label={0: "SAFE", 1: "VULNERABLE"},
    label2id={"SAFE": 0, "VULNERABLE": 1}
)

print(f"✅ Модель загружена. Параметров: {sum(p.numel() for p in model.parameters()):,}")


print("\n" + "=" * 60)
print("СОЗДАНИЕ ДАТАСЕТОВ")
print("=" * 60)

class CodeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

train_dataset = CodeDataset(train_encodings, train_labels)
val_dataset = CodeDataset(val_encodings, val_labels)


print("\n" + "=" * 60)
print("НАСТРОЙКА МЕТРИК")
print("=" * 60)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)


    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None, zero_division=0
    )


    f1_vulnerable = f1[1] if len(f1) > 1 else 0


    report = classification_report(labels, predictions,
                                   target_names=['Безопасный', 'Уязвимый'],
                                   output_dict=True, zero_division=0)

    return {
        'accuracy': accuracy,
        'f1_vulnerable': f1_vulnerable,
        'precision_safe': precision[0],
        'recall_safe': recall[0],
        'f1_safe': f1[0],
        'precision_vulnerable': precision[1],
        'recall_vulnerable': recall[1],
        'vulnerable_support': int(labels.sum()),
        'predicted_vulnerable': int(predictions.sum())
    }


print("\n" + "=" * 60)
print("НАСТРОЙКА ОБУЧЕНИЯ")
print("=" * 60)

training_args = TrainingArguments(
    output_dir='./balanced_training',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1_vulnerable",
    greater_is_better=True,
    push_to_hub=False,
    report_to='none',
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


print("\n" + "=" * 60)
print("НАЧАЛО ОБУЧЕНИЯ")
print("=" * 60)

print("⚠️ Обучение займет 15-20 минут...")
trainer.train()


print("\n" + "=" * 60)
print("ОЦЕНКА МОДЕЛИ")
print("=" * 60)

eval_results = trainer.evaluate()
print("\n📊 РЕЗУЛЬТАТЫ ОЦЕНКИ:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")


print("\n" + "=" * 60)
print("ТЕСТИРОВАНИЕ НА ПРИМЕРАХ")
print("=" * 60)

test_examples = [

    ("void vulnerable_gets() { char buf[10]; gets(buf); }", "🔴 gets() - критическая уязвимость"),
    ("void vulnerable_strcpy() { char dest[10]; strcpy(dest, src); }", "🔴 strcpy() без проверки"),
    ("int vulnerable_scanf() { char buf[10]; scanf(\"%s\", buf); return 0; }", "🔴 scanf(%s) опасен"),


    ("void safe_fgets() { char buf[10]; fgets(buf, 10, stdin); }", "🟢 fgets() с лимитом"),
    ("void safe_strncpy() { char dest[10]; strncpy(dest, src, 9); dest[9] = 0; }", "🟢 strncpy() с ограничением"),
    ("int safe_scanf() { char buf[10]; scanf(\"%9s\", buf); return 0; }", "🟢 scanf() с шириной"),
]

print("\n🔍 ТЕСТИРОВАНИЕ МОДЕЛИ НА РАЗНЫХ ПРИМЕРАХ:")

model.eval()

for i, (code, description) in enumerate(test_examples, 1):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, max_length=256)

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    vuln_prob = predictions[0][1].item() * 100
    safe_prob = predictions[0][0].item() * 100


    if vuln_prob > 70:
        status = "🔴 ВЫСОКИЙ РИСК УЯЗВИМОСТИ"
        color = "red"
    elif vuln_prob > 40:
        status = "🟡 СРЕДНИЙ РИСК"
        color = "orange"
    else:
        status = "🟢 НИЗКИЙ РИСК / БЕЗОПАСНО"
        color = "green"

    print(f"\n{'='*50}")
    print(f"Пример {i}: {description}")
    print(f"{'='*50}")
    print(f"Код: {code}")
    print(f"\nРезультат анализа:")
    print(f"  Статус: {status}")
    print(f"  Вероятность уязвимости: {vuln_prob:.1f}%")
    print(f"  Вероятность безопасности: {safe_prob:.1f}%")


    patterns = {
        'gets(': '🚨 КРИТИЧЕСКО: gets() запрещена! Используйте fgets()',
        'strcpy(': '⚠️ ОПАСНО: strcpy() без проверки длины. Используйте strncpy()',
        'scanf("%s': '⚠️ РИСК: scanf() с %s. Ограничьте ширину (%10s)',
        'strcat(': '⚠️ ВНИМАНИЕ: strcat() может переполнить буфер',
        'system(': '⚠️ ОПАСНО: system() может выполнить произвольные команды',
    }

    detected = []
    for pattern, warning in patterns.items():
        if pattern in code:
            detected.append(warning)

    if detected:
        print(f"\n🔎 Обнаружены паттерны:")
        for warn in detected:
            print(f"  • {warn}")



save_path = "./code_vulnerability_detector_final"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)




In [ ]:


import gradio as gr
import torch
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "./code_vulnerability_detector_final"

class SmartVulnerabilityDetector:
    def __init__(self):
        print(" Инициализация детектора уязвимостей...")

        try:

            self.tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
            self.model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
            self.model.eval()
            print("✅ ML модель загружена")


            print(f"  Классы: {self.model.config.id2label}")

        except Exception as e:
            print(f"⚠️ Не удалось загрузить ML модель: {e}")
            print("Используется rule-based детектор")
            self.model = None
            self.tokenizer = None

    def analyze_with_ml(self, code):
        """Анализ с помощью ML модели"""
        try:
            inputs = self.tokenizer(
                code[:500],
                return_tensors="pt",
                truncation=True,
                max_length=256,
                padding=True
            )

            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)


            vuln_prob = probs[0][1].item() * 100

            return {
                "ml_vulnerability_score": vuln_prob,
                "ml_safety_score": 100 - vuln_prob,
                "raw_probs": probs[0].tolist()
            }

        except Exception as e:
            print(f"Ошибка ML анализа: {e}")
            return None

    def analyze_with_rules(self, code):
        """Rule-based анализ (работает всегда!)"""
        score = 0
        warnings = []
        patterns_found = []


        critical_patterns = {
            r'gets\s*\(': {
                'weight': 70,
                'message': '🚨 КРИТИЧЕСКАЯ УЯЗВИМОСТЬ: функция gets() ЗАПРЕЩЕНА!'
            },
            r'system\s*\([^)]*\)': {
                'weight': 60,
                'message': '🚨 ВЫСОКИЙ РИСК: system() выполняет произвольные команды'
            },
            r'strcpy\s*\(': {
                'weight': 55,
                'message': '⚠️ ВЫСОКИЙ РИСК: strcpy() без проверки длины'
            },
            r'scanf\s*\([^)]*%s': {
                'weight': 50,
                'message': '⚠️ ВЫСОКИЙ РИСК: scanf() с %s без ограничения'
            },
        }


        medium_patterns = {
            r'strcat\s*\(': {
                'weight': 40,
                'message': '⚠️ СРЕДНИЙ РИСК: strcat() может переполнить буфер'
            },
            r'sprintf\s*\([^)]*%[^)]*\)': {
                'weight': 35,
                'message': '⚠️ РИСК: sprintf() может привести к переполнению'
            },
            r'malloc\s*\([^)]*\)[^{}]*$': {
                'weight': 30,
                'message': '⚠️ ВНИМАНИЕ: malloc() без соответствующего free()'
            },
            r'printf\s*\([^)]*%[^f]': {
                'weight': 25,
                'message': '⚠️ ВНИМАНИЕ: printf() с переменным форматом'
            },
        }


        for pattern, info in critical_patterns.items():
            if re.search(pattern, code, re.IGNORECASE):
                score += info['weight']
                warnings.append(info['message'])
                patterns_found.append(pattern)


        for pattern, info in medium_patterns.items():
            if re.search(pattern, code, re.IGNORECASE):
                score += info['weight']
                warnings.append(info['message'])
                patterns_found.append(pattern)


        safe_patterns = {
            r'fgets\s*\(': {
                'weight': -20,
                'message': '✅ БЕЗОПАСНО: fgets() с ограничением размера'
            },
            r'strncpy\s*\(': {
                'weight': -15,
                'message': '✅ БЕЗОПАСНО: strncpy() с указанием длины'
            },
            r'free\s*\(': {
                'weight': -10,
                'message': '✅ ПРАВИЛЬНО: освобождение памяти'
            },
        }

        for pattern, info in safe_patterns.items():
            if re.search(pattern, code, re.IGNORECASE):
                score = max(0, score + info['weight'])
                warnings.append(info['message'])


        probability = min(100, score)


        if probability >= 60:
            risk_level = "🔴 КРИТИЧЕСКИЙ"
            color = "#ff4444"
        elif probability >= 40:
            risk_level = "🟡 ВЫСОКИЙ"
            color = "#ffaa00"
        elif probability >= 20:
            risk_level = "🟠 СРЕДНИЙ"
            color = "#ff8800"
        else:
            risk_level = "🟢 НИЗКИЙ"
            color = "#00cc66"

        return {
            "rule_vulnerability_score": probability,
            "risk_level": risk_level,
            "color": color,
            "warnings": warnings if warnings else ["✅ Опасных паттернов не обнаружено"],
            "patterns_found": patterns_found if patterns_found else ["Нет опасных паттернов"]
        }

    def get_recommendations(self, code):
        """Генерация рекомендаций по исправлению"""
        recommendations = []
        code_lower = code.lower()


        if 'gets(' in code_lower:
            recommendations.append("""
            **🚨 КРИТИЧЕСКАЯ УЯЗВИМОСТЬ: gets()**
            ```c
            // ВМЕСТО ЭТОГО:
            char buffer[10];
            gets(buffer);

            // ИСПОЛЬЗУЙТЕ ЭТО:
            char buffer[10];
            fgets(buffer, sizeof(buffer), stdin);
            // Удалите символ новой строки если нужно
            buffer[strcspn(buffer, "\\n")] = 0;
            ```
            """)


        if 'strcpy(' in code_lower and 'strncpy' not in code_lower:
            recommendations.append("""
            **⚠️ ОПАСНО: strcpy() без проверки длины**
            ```c
            // ВМЕСТО ЭТОГО:
            char dest[10];
            strcpy(dest, src);

            // ИСПОЛЬЗУЙТЕ ЭТО:
            char dest[10];
            strncpy(dest, src, sizeof(dest) - 1);
            dest[sizeof(dest) - 1] = '\\0';
            ```
            """)


        if 'scanf(' in code_lower and '%s' in code:
            recommendations.append("""
            **⚠️ РИСК: scanf() с неограниченным вводом**
            ```c
            // ВМЕСТО ЭТОГО:
            char buffer[10];
            scanf("%s", buffer);

            // ИСПОЛЬЗУЙТЕ ЭТО:
            char buffer[10];
            scanf("%9s", buffer);  // Ограничение ширины

            // ИЛИ ЛУЧШЕ:
            fgets(buffer, sizeof(buffer), stdin);
            ```
            """)

        if 'malloc(' in code_lower and 'free(' not in code_lower:
            recommendations.append("""
            **⚠️ УТЕЧКА ПАМЯТИ: malloc() без free()**
            ```c
            // ВМЕСТО ЭТОГО:
            int *arr = malloc(n * sizeof(int));
            // ... использование ...

            // ИСПОЛЬЗУЙТЕ ЭТО:
            int *arr = malloc(n * sizeof(int));
            if (arr == NULL) {
                // Обработка ошибки
            }
            // ... использование ...
            free(arr);  // Не забудьте освободить!
            ```
            """)

        if not recommendations:
            recommendations.append("""
            **✅ РЕКОМЕНДАЦИИ:**
            - Код соответствует best practices безопасности
            - Продолжайте использовать безопасные функции
            - Регулярно проводите статический анализ кода
            """)

        return recommendations

    def analyze(self, code, threshold=30):
        """Основная функция анализа (гибридный подход)"""
        if not code or len(code.strip()) < 10:
            return {"error": "❌ Введите код длиннее 10 символов"}

        print(f"\n📝 Анализируем код ({len(code)} символов):")
        print("-" * 50)


        rule_result = self.analyze_with_rules(code)


        ml_result = None
        if self.model:
            ml_result = self.analyze_with_ml(code)


        if ml_result and ml_result["ml_vulnerability_score"] > 0:

            final_score = (
                ml_result["ml_vulnerability_score"] * 0.6 +
                rule_result["rule_vulnerability_score"] * 0.4
            )
            method = " ML +  Rules Hybrid"
            ml_details = {
                "ml_score": f"{ml_result['ml_vulnerability_score']:.1f}%",
                "safety_score": f"{ml_result['ml_safety_score']:.1f}%"
            }
        else:

            final_score = rule_result["rule_vulnerability_score"]
            method = " Rule-Based Analysis"
            ml_details = {}


        if final_score >= threshold:
            final_status = "🔴 УЯЗВИМЫЙ"
            final_color = "#ff4444"
        else:
            final_status = "🟢 БЕЗОПАСНЫЙ"
            final_color = "#00cc66"


        recommendations = self.get_recommendations(code)


        result = {
            "status": final_status,
            "status_color": final_color,
            "final_vulnerability_score": f"{final_score:.1f}%",
            "risk_level": rule_result["risk_level"],
            "analysis_method": method,
            "code_info": {
                "lines": code.count('\n') + 1,
                "size": f"{len(code)} символов",
                "language": "C/C++"
            },
            "warnings": rule_result["warnings"],
            "patterns_found": rule_result["patterns_found"],
            "recommendations": recommendations,
            "threshold_used": f"{threshold}%"
        }


        if ml_details:
            result.update(ml_details)

        return result


detector = SmartVulnerabilityDetector()


EXAMPLES = [
    ["void dangerous() { char buffer[10]; gets(buffer); }"],
    ["void safe_input() { char name[50]; fgets(name, 50, stdin); }"],
    ["void bad_copy() { char dest[10]; strcpy(dest, src); }"],
    ["void good_copy() { char dest[10]; strncpy(dest, src, 9); dest[9] = 0; }"],
    ["int risky() { char cmd[100]; system(\"ls -la\"); return 0; }"],
]

def create_interface():
    with gr.Blocks(
        title="Детектор уязвимостей в коде",
        theme=gr.themes.Soft(),
        css="""
        .critical { color: #ff4444 !important; font-weight: bold; }
        .warning { color: #ffaa00 !important; font-weight: bold; }
        .safe { color: #00cc66 !important; font-weight: bold; }
        .code-example { font-family: monospace; background: #f5f5f5; padding: 10px; border-radius: 5px; }
        """
    ) as demo:


        gr.Markdown("""
        # Детектор уязвимостей в программном коде


        ---
        """)


        with gr.Row():
            with gr.Column(scale=2):
                code_input = gr.Textbox(
                    label="Введите код на C/C++ для анализа",

                    lines=15,
                    elem_id="code_input"
                )

                threshold_slider = gr.Slider(
                    minimum=10,
                    maximum=50,
                    value=30,
                    step=5,
                    label=" Порог чувствительности",
                    info="При пороге 30%: если вероятность уязвимости > 30%, код считается уязвимым"
                )

                with gr.Row():
                    analyze_btn = gr.Button(
                        "Анализировать код",
                        variant="primary",
                        size="lg",
                        elem_id="analyze_btn"
                    )
                    clear_btn = gr.Button("Очистить", variant="secondary")

            with gr.Column(scale=1):
                output_json = gr.JSON(
                    label="Результаты анализа",
                    elem_id="results"
                )


        gr.Markdown("### 🧪 Примеры для тестирования")
        gr.Examples(
            examples=EXAMPLES,
            inputs=[code_input],
            label="Нажмите на пример для быстрой проверки",
            cache_examples=False
        )



        def analyze_handler(code, threshold):
            return detector.analyze(code, threshold)

        def clear_handler():
            return ""

        analyze_btn.click(
            fn=analyze_handler,
            inputs=[code_input, threshold_slider],
            outputs=[output_json]
        )

        clear_btn.click(
            fn=clear_handler,
            inputs=[],
            outputs=[code_input]
        )

    return demo


if __name__ == "__main__":
    print("Запуск детектора уязвимостей...")
    demo = create_interface()
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False,
        debug=True
    )